In [ ]:
import torch
import numpy as np
import cv2
from time import time
class yolov5_objectDetection:
    """
    This class implements Yolov5 model to detect certain objects in a specified video using Opencv2.
    """
#####
    def __init__(self, direct, output_file="saved_video.avi"):
        """
        Initializing the class with specified directory and output file.
        direct: speicified directory from the computer
        output_file: output viode's file name.
        """
        self._directory = direct
        self.model = self.loading_model()
        self.model.classes = [2, 3, 5]   #only car, motorcycle and bus
        self._classes = self.model.names
        self.output_file = output_file
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

#####
    def get_video_from_directory(self):
        """
        Creates a new video inputted by a video in specified directory. 
        returns video capture object having the lowest quality frame avaiilable for video.
        """
        return cv2.VideoCapture(self._directory)

#####  
    def loading_model(self):
        """
        this command load the Yolo5 model from pytorch hub.
        returns the pretrained Pytorch model.
        """
        yolov5_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
        return yolov5_model
#####  
    def score_label_frame(self, single_frame):
        """
        Taking a frame as input, and this frame is labelled using yolov5 model.
        single_frame: input frame 
        return: Coordinates and score labels of objects in the frame detected by yolov5 odel.
        """
        self.model.to(self.device)
        single_frame = [single_frame]
        results = self.model(single_frame)
        labels, coordinates = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()
        return labels, coordinates

#####
    def class_to_label(self, score):
        """
        this function converts the score value of label to corresponding label: car, motorcycle, bus
        score: numeric score
        return: corresponding label
        """
        return self._classes[int(score)]
    
    def plot_boxes(self, results, single_frame, max_lane_cord):
        """
        results: labels and coordinates predicted by model on a single frame
        frame: scored Frame .
        return: new Frame having bounding boxes and labels in it
        """
        labels, coordinates = results
        n = len(labels)
        x_shape, y_shape = single_frame.shape[1], single_frame.shape[0]
        for i in range(n):
            rows = coordinates[i]
            if rows[4] >= 0.2:
                x1, y1, x2, y2 = int(rows[0]*x_shape), int(rows[1]*y_shape), int(rows[2]*x_shape), int(rows[3]*y_shape)
                green = (0, 255, 0)
                red = (0, 0, 255)
                if np.mean([x1,x2]) <= max_lane_cord:
                    cv2.rectangle(single_frame, (x1, y1), (x2, y2), green, 4)
                else:
                    cv2.rectangle(single_frame, (x1, y1), (x2, y2), red, 4)
                cv2.putText(single_frame, self.class_to_label(labels[i]), (x1, y1), cv2.FONT_HERSHEY_PLAIN, 0.9, (255, 120, 90), 2) 
        return single_frame
    def __call__(self):
        """
        This function is to execute the class, it runs by certain loops to read a video frame by frame,
        and save out the output into a new file.
        """
        player = self.get_video_from_directory()
        x_shape = int(player.get(cv2.CAP_PROP_FRAME_WIDTH))
        y_shape = int(player.get(cv2.CAP_PROP_FRAME_HEIGHT))
        four_cc = cv2.VideoWriter_fourcc(*"MJPG")
        out = cv2.VideoWriter(self.output_file, four_cc, 20, (x_shape, y_shape))
        while True:
            start_time = time()
            ret, single_frame = player.read()
            # frame_copy = single_frame.copy()
            if not ret:
                print("Exiting ...")
                break
            if cv2.waitKey(1) == ord('q'):
                break

            frame_ar = np.asarray(single_frame)
            stencil = np.zeros_like(frame_ar[:,:,0]) 
            # trapezoid = np.array([[250, y_shape], [830, 720], [1000, 720], [1670, y_shape]])
            trapezoid = np.array([[170, 700], [520, 500], [940, 470], [1300, 740]])
            cv2.fillConvexPoly(stencil, trapezoid, 1)
            img = cv2.bitwise_and(frame_ar[:,:,0], frame_ar[:,:,0], mask=stencil)
            ret, thresh = cv2.threshold(img, 100, 135, cv2.THRESH_BINARY)
            lines = np.array(cv2.HoughLinesP(thresh, 1, np.pi/180, 40, maxLineGap=200))
            lines = lines.reshape(lines.shape[0], lines.shape[2])
            
            for line in lines:
                x1, y1, x2, y2 = line
                cv2.line(single_frame, (x1, y1), (x2, y2), (255, 255, 255), 3)
            
            results = self.score_label_frame(single_frame)
            single_frame = self.plot_boxes(results, single_frame, lines[0,:].max())
            
            end_time = time()
            fps = 1/np.round(end_time - start_time, 3)
            print(f"Frames Per Second : {fps}")
            out.write(single_frame)
            if cv2.waitKey(1) == ord('q'):
                break
# Creating a new object and executing.
a = yolov5_objectDetection("/Users/malisarsil/Desktop/test_video.mp4", "saved_video.avi")
a()